# Signpost Article Views

We on the editorial board of the *Wikipedia Signpost* have wanted for a while now to run statistics on the page views that our stories generate. Understanding our distribution helps us understand our readership, and understanding our readership helps us to direct our energies in our future writing towards the stories that our readers most want to see and read. Distribution management is a basic tenet of newspaper management (I know of at least one "recovering academic" working in circulation analytics at *The New York Times*), but unfortunately one we've never quite been able to do before.

In the past the only way to get ever-popular pageview statistics was to use [stats.grok.se](http://stats.grok.se/), a venerable website run by an involved community member which did define an API for query but which, alas, didn't have the throughput to serve the results in reasonable time. It relied on huge database dumps which are a pain to download, let alone parse. Finally (belatedly) acting on this fact, the Wikimedia Foundation in December 2015 [rolled out a full-suite Pageview RESTBase API](http://blog.wikimedia.org/2015/12/14/pageview-data-easily-accessible/) getting view statistics on pages on Wikipedia has become worlds easier, especially with the R and Python API clients also released for the purpose.

**Caveat**: The dataset for this analysis is the set of all *Signpost* article views from the period 2015-10-07 to 2015-12-30. The *Wikipedia Signpost* is published on a weekly basis (and has been since its inception in 2005): this period saw 12 publications (one issue was skipped around Christmas-time). We would like to extend the analysis further back, but due to differing standard of reporting in older applications and logs, for the moment early October 2015 is the earliest date for which Pageview API data is available.

## Initialization code

This notebook uses [pandas](http://pandas.pydata.org/) for tabulation and [mwapi](https://github.com/mediawiki-utilities/python-mwapi) plus [mwviews](https://github.com/mediawiki-utilities/python-mwviews) for querying.

In [1]:
from pageviews import PageviewsClient
import arrow
import datetime
import urllib
from pandas import DataFrame
from pandas import Series
import pandas as pd
import mwapi
import requests


def viewcounts(article_name, start=None, end=None):
    """
    Fetches the viewcounts.
    """
    article_name = article_name.replace(' ', '_')
    parsed_article_name = urllib.parse.quote(article_name).replace('/', '%2F')
    p = PageviewsClient().article_views("en.wikipedia",
                                        [parsed_article_name],
                                        access="all-access",
                                        # access="users",
                                        granularity="daily",
                                        start=start,
                                        end=end)
    counts = {key: p[key][article_name] for key in p.keys()}
    return [p[key][article_name] for key in sorted(p.keys())]
    # TODO: Fix be pre-padding with 0s if output is below 15 length.

def article_viewcounts(article_name):
    """
    Fetches a list of the Signpost article viewcount from the date of the publication window.
    The Signpost is usually published late, so a generous 14 day news "cycle" is allotted as the publication window.
    In reality views are low before publication and after publication of the next issue, so this doesn't have much effect.
    """
    pubdate = arrow.get(article_name.split("/")[1])
    enddate = (pubdate + datetime.timedelta(days=14)).strftime('%Y%m%d%H')
    pubdate = pubdate.strftime('%Y%m%d%H')
    return viewcounts(article_name, start=pubdate, end=enddate)

def total_viewcount(article_name):
    """
    Returns the total 60-day viewcount.
    """
    return sum(article_viewcounts(article_name))

def average_daily_viewcount(article_name):
    """
    Returns the average daily viewcount of the article.
    """
    counts = article_viewcounts(article_name)
    return sum(counts)/len(counts)

def get_all_articles(prefix):
    """
    Returns a list of the titles of all of the Signpost articles published after a certain prefix.
    Prefix is 2015-10-07 for now, the earliest published Signpost story for which data is available (yet).
    """
    session = mwapi.Session('https://en.wikipedia.org', user_agent='signpostviews Jupyter notebook')
    raw_result = session.get(action='query',
                             list='allpages',
                             apfrom=prefix,
                             apto='Wikipedia Signpost/A',
                             apprefix='Wikipedia Signpost',
                             apnamespace=4,
                             aplimit=500,
                             formatversion=2)
    # The >= 2 call filters out results e.g. Wikipedia:Wikipedia Signpost/2015-07-18
    # The not 2016 call keeps out titles too recent to have full data for.
    result = [r['title'] for r in raw_result['query']['allpages'] if r['title'].count("/") >= 2]
    return result

def tabulate(articles):
    pass_dict = {article: article_viewcounts(article) for article in articles}
    return pass_dict

In [2]:
# Get all of the articles published in the period 2015-10-07 to 2016-01-01.
targets = [article for article in get_all_articles("Wikipedia Signpost/2015-10-07/Op-ed") if '2016' not in article]
# Create the values dict that should be passed to the DataFrame for initialization.
all_views = tabulate(targets)

In [3]:
pd.set_option('display.max_rows', None)
# Initialize the frame using the data from the previous step.
frame = DataFrame([all_views[key] for key in sorted(all_views.keys())],
                  index=sorted(all_views.keys()))
# Fill missing values (not enough views to be logged so the API returns NaN) with 0.
frame = frame.fillna(0)
# Compute some per-page summary statistics.
# frame['avg'] = frame.apply(lambda x: int(sum(x) / 15), axis=1) # Not that useful because of the many empty values.
frame['total'] = frame.apply(lambda x: sum(x), axis=1)

In [4]:
# Drop entries which obviously never made it to publication.
frame = frame[frame['total'] > 200]
# Fix a particular trouble spot, where an article was renamed post-publication.
frame.ix['Wikipedia:Wikipedia Signpost/2015-10-14/WikiConference report'] = frame.ix['Wikipedia:Wikipedia Signpost/2015-10-14/WikiConference report'] + frame.ix['Wikipedia:Wikipedia Signpost/2015-10-14/WikiConference Report']
frame = frame.drop('Wikipedia:Wikipedia Signpost/2015-10-14/WikiConference Report')

In [5]:
def get_title(pub):
    """
    Parses a raw HTTP request to fetch the title of the article based on its URL, and returns it.
    """
    ret = requests.get("http://en.wikipedia.org/wiki/" + pub).text
    # return ret
    ret.find('<span class="mw-headline"')
    ret = ret[ret.find('<span class="mw-headline"'):]
    ret = ret[:ret.find('</span></h2>')]
    ret = ret[ret.find('>') + 1:]
    return ret

In [6]:
# At this point the frame is indexed by strings of the form e.g. "Wikipedia Signpost/2015-07-11/News and notes".
# Refactor the `DataFrame` to give us more useful information (in particular page titles).
frame['type'] = [title.split("/")[2] for title in frame.index]
frame['date'] = [title.split("/")[1] for title in frame.index]
frame['title'] = [get_title(title) for title in frame.index]
frame = frame.set_index('title')
frame.index.name=None

In [7]:
# Create a multiindex to level-store date -> title, using the 'date' column created as a part of the previous step.
multiindex = pd.MultiIndex.from_arrays([frame['date'], frame.index])
frame.index = multiindex
frame.index.names = ['date', 'title']
del frame['date']

In [8]:
# Refactor the columns so that type is in front instead of in the back.
cols = frame.columns.tolist()
cols = cols[-1:] + cols[:-1]
frame = frame[cols]

## Raw Data

The following table contains the raw data of all *Signpost* article views from the period 2015-10-07 to 2015-12-30. In these 12 issues the *Signpost* published 86 individual stories. The dates correspond with days after the official publication date (0 means day of publication), with a long "after" time in order to account for the variability in *Signpost* publication time: due to editorial constrains, in this period copies were usually published about two days after the actual post date.

In [9]:
frame

type  \
date       title                                                                                                                                                                     
2015-10-07 Walled gardens of corruption                                                                                                                                      Op-ed   
           Tech news in brief                                                                                                                                    Technology report   
           Reality is for losers                                                                                                                                    Traffic report   
2015-10-14 Third Wikimedia Spain conference takes place in Madrid                                                                                                             Blog   
           Why the news media needs a Wikipedian in residence                                                                                                            Editorial   
           A fistful of dollars                                                                                                                                   Featured content   
           2015–2016 Q1 fundraising update sparks mailing list debate                                                                                               News and notes   
           WikiConference USA 2015: built on good faith                                                                                                                      Op-ed   
           Tech news in brief                                                                                                                                    Technology report   
           Screens, sport, Reddit, and death                                                                                                                        Traffic report   
           US gathering sees speeches from Andrew Lih, AfroCrowd, and the Archivist of the United States                                                     WikiConference report   
2015-10-21 Four ArbCom cases ongoing                                                                                                                            Arbitration report   
           Women and Wikipedia: the world is watching                                                                                                                    Editorial   
           A more balanced week                                                                                                                                   Featured content   
           "Wikipedia's hostility to women"                                                                                                                           In the media   
           Wikimedia lawsuit against NSA dismissed; Affiliates mailing list launched                                                                                News and notes   
           Wikipedia is significantly amplifying the impact of open-access publications                                                                                      Op-ed   
           One year of GamerGate, or how I learned to stop worrying and love bare rule-level consensus                                                              Special report   
           Tech news in brief                                                                                                                                    Technology report   
           Hiding under the covers of the Internet                                                                                                                  Traffic report   
2015-10-28 A second attempt at Arbitration enforcement                                                                                                          Arbitratio

The *Wikipedia Signpost* is considered the [newspaper of record](https://en.wikipedia.org/wiki/Newspaper_of_record) within the internal Wikimedian community (one kind reader I spoke to in person once excitedly called it "the *The New York Times* of Wikipedia"), but within individual issues different stories have different lifetimes, depending on how interested the readers are and on where they get reposted, and so some stories have more and longer impact periods than others. The hard cut-off is often the publication of the next *Signpost* issue, but for the best stories discussion and reverb can continue for a week or even more afterwards.

Some caveats:

* The data here is likely a very slight undercount (due to the effects described above).

* Stories are occasionally summarized or linked to in more mainstream media or within major discussions within the project long after the fact of publication, which can produce sudden and very strong one-day spikes in viewership. Such spikes are not captured within this data for reason of expense (besides, they're not really in the current news cycle anymore).

* Stories are occasionally used as references in other pages, reread by community members searching through the *Signpost* archives, or clicked through to by "see more" content within current articles. These effects are also not captured.

## Summary statistics

**How many views does the average *Signpost* story get?**

In [10]:
int(round(sum(frame['total'])/len(frame['total']), 0))

1550

**What is the standard deviation?**

[Standard deviation](https://en.wikipedia.org/wiki/Standard_deviation) is a measure of the variability of a dataset: here, the variability of the number of views that a *Signpost* article begets.

In [11]:
int(round(frame['total'].describe()['std']))

730

As you can see the standard deviation of a *Signpost* story is extrordinary! Taken as a percentage of the average:

In [12]:
round(frame['total'].describe()['std'] / (sum(frame['total'])/len(frame['total'])), 6)

0.470997

This says the variation in the number of views that a story can get is very high, and hints at the (intuitive, but never before so clearly stated) fact that different section of the *Signpost* are on different magnitudes of viewership.

It is more meaningful to examine indiviudal sections, not the *Signpost* as a whole.

**What is the average number of stories per issue?**

In [13]:
issue_sizes = [len(frame.ix[date]) for date in list(set(frame.index.get_level_values(0)))]
float('{0:.3f}'.format(sum(issue_sizes) / len(issue_sizes)))

7.167

**What was the most popular story? The least popular one?**

Keep in mind that we are working with a small slice of *Signpost* publication history, just in the range October to December 2015 (the *Signpost* has been published continuously since early 2005). So these are not all-time values, just the ones within our dataset.

In [14]:
print("Most popular:", '"' + frame[frame['total'] == frame['total'].max()].index[0][0] + '"', "with", int(frame['total'].max()), "views")
print("Least popular:", '"' + frame[frame['total'] == frame['total'].min()].index[0][0] + '"', "with", int(frame['total'].min()), "views")
print("This is a difference of", int(frame['total'].max() - frame['total'].min()), "views!")

Most popular: "2015-12-30" with 5666 views
Least popular: "2015-10-28" with 849 views
This is a difference of 4817 views!


The most popular story within the dataset is [a *Signpost* story on the sudden removal of community-elected trustee James Heilman from the Wikimedia Foundation's governing body](http://en.wikipedia.org/wiki/Wikipedia:Wikipedia Signpost/2015-12-30/News and notes), the Board of Trustees, an unprecedented and terrifically opaque move the fallout from which continues to dominate the conversation within the movement in the weeks thereafter. Note that this data is also cut off, so this number is likely a fairly sizable undercount.

The least popular story is a [Tech/News republication](http://en.wikipedia.org/wiki/Wikipedia:Wikipedia Signpost/2015-10-28/Technology report) which just happens to have the unfortunate fact of being the least popular article in the least popular section.

**What are the top five most popular stories?**

Keep in mind that we are working with a small slice of *Signpost* publication history, just in the range October to December 2015.

In [15]:
frame.sort('total', ascending=False).head(5).ix[:, ['type', 'total']]

,,type,total
date,title,,
2015-12-30,WMF Board dismisses community-elected trustee,News and notes,5666
2015-10-21,Women and Wikipedia: the world is watching,Editorial,4069
2015-12-02,Whither Wikidata?,Op-ed,4041
2015-12-09,Wikidata: Knowledge from different points of view,Op-ed,2930
2015-11-18,ArbCom election—candidates’ opinions analysed,Special report,2907


## By section

**How does popularity break down by section?**

Quick notes on what the section are:

* The "Technology report" used to be a specialized report for the tech crowd, however it has not had a regular writer for a long time and so has now been converted to a script-based republication of the similar but bare-bones [Tech/News](https://meta.wikimedia.org/wiki/Tech/News) report. Since many will have originally read it there first anyway, this is likely the least popular section, just as a matter of course.
* The "Gallery" section is a composition of images, either from photo competitions or from meetups or, occasionally, just to go with the season.
* The "Blog" section selectively republished reports first posted the [Wikimedia Blog](http://blog.wikimedia.org/).
* The "Traffic report" speculates on the popularity of the top ten most popular articles on Wikipedia of the week. It is light-hearted material that is well-read for what it is.
* "Recent research" (also compiled separately as the [Recent Research report](https://meta.wikimedia.org/wiki/Research:Newsletter)) is a monthly section which summarizes the state of academic research on Wikipedia and the Wikimedias. It is thus a dense read, for good reason!
* The "Arbitration report" summarizes the outcomes and progress in cases took on by the [Arbitration Committee](https://en.wikipedia.org/wiki/Wikipedia:Arbitration_Committee), the community court of last resort.
* The "WikiProject report" interviews the active members of a WikiProject such a WikiProject Volcanoes or WikiProject Iceland.
* "In the media" summarizes the week's media mentions of Wikipedia and the Wikimedia movement.
* "News and notes" summarizes the week's internal news specific to the operations of the encyclopedia and the Wikimedia movement.
* "Opinion" is when someone sticks their nose out and stakes a case in the *Signpost*, arguing for or against something. It's no wonder it's well-read!
* The "Special report" is a well-researched *Signpost* inside scoop on something of the project, and is a report which is more just-the-facts than an opinion column.

In [16]:
def views(section_title):
    """
    Fetch a list of view counts for a particular section.
    """
    return [frame.ix[article, 'total'] for article in frame.index if section_title in frame.ix[article, 'type']]

def avg_views(section_title):
    """
    Fetch the average view count for a particular section.
    """
    v = views(section_title)
    return int(round(sum(v) / len(v)))


ind = ['News and notes',
       'In the media',
       'Technology report',
       'Traffic report',
       'Arbitration report',
       'WikiProject report',
       'Recent research',
       'Blog',
       'Gallery',
       'Opinion',
       'Special report']
viewf = DataFrame([avg_views('News and notes'),
                   avg_views('In the media'),
                   avg_views('Technology report'),
                   avg_views('Traffic report'),
                   avg_views('Arbitration report'),
                   avg_views('WikiProject report'),
                   avg_views('Recent research'),
                   avg_views('Blog'),
                   avg_views('Gallery'),
                   # Opinion
                   int(round(sum(views('Op-ed') + views('Editorial')) / len(views('Op-ed') + views('Editorial')))),
                   # Special
                   int(round(sum(views('Special report') + views('In focus')) / len(views('Special report') + views('In focus'))))],
                   index=ind,
                   columns=['average']).sort('average', ascending=False)
viewf

,average
Opinion,2303
Special report,2243
News and notes,2155
In the media,1679
WikiProject report,1600
Arbitration report,1545
Recent research,1422
Traffic report,1251
Blog,1163
Gallery,1073


These results are not exactly surprising, but it's nice to have concrete numbers at last!

## Publication cycles

The *Signpost* has been published on a near-continuous weekly basis since early 2005. An interesting question to ask is one of publication cycles.

In this context a **publication cycle** is one start-to-finish *Signpost* issue: from the initial drafts produced by the sections' writers to the polished final copy that is published for all to read. The **news cycle** is almost-but-not-quite-weekly accumulation of goings-on that is covered in each *Signpost* issue - that is, the space between one issue and the next. The *Signpost* is variably published usually a day or two late in order to accomadate the schedules of the publishers and also to best fit whatever the current news cycle may be, and so as a result the **post date** that each issue is labelled with (`2015-10-07`, for instance) is very rarely the actual **publication date** (which was `2015-10-09` in this example case).

All of the data above ignored cycles: it included page views a little before and a little after the story was live. We now recalibrate the data somewhat with additional information to get a better picture of what a publication cycle is like. This allows us to study **in-cycle views**, that is, views which occur while the story is front page copy.

**How late is the *Signpost* published?**

Oh bugger. The following chart tabulates post date, publication date, the number of days of difference between the two, and the resultant length of time for which the current stories stay on the front page (the length of the news cycle).

In [17]:
# Get unique copy dates.
# This is a small enough sample (12 stories) that we can just hand-scrape the information off of the main page:
# https://en.wikipedia.org/w/index.php?title=Wikipedia:Wikipedia_Signpost&action=history
lag = list(zip([4, 4, 4, 4, 3, 4, 3, 3, 4, 3, 3, 0], sorted(list(set(frame.index.get_level_values(0))))))
lateness = DataFrame([l[0] for l in lag], index=[l[1] for l in lag], columns=["lateness"])
lateness.index.name = "post date"
# Let's also tabulate the actual dates.
lateness['publication date'] = [(arrow.get(post_date) + datetime.timedelta(days=int(lateness.ix[post_date, 'lateness']))).format("YYYY-MM-DD") for post_date in lateness.index]
lateness = lateness[['publication date', 'lateness']]
# Let's also tabulate the length of the resultant news cycles.
lateness['news cycle length'] = [7 + lateness.iloc[ind + 1]['lateness'] - lateness.iloc[ind]['lateness'] for ind in range(0, len(lateness.index) - 1)] + [7]
lateness

,publication date,lateness,news cycle length
post date,,,
2015-10-07,2015-10-11,4,7
2015-10-14,2015-10-18,4,7
2015-10-21,2015-10-25,4,7
2015-10-28,2015-11-01,4,6
2015-11-04,2015-11-07,3,8
2015-11-11,2015-11-15,4,6
2015-11-18,2015-11-21,3,7
2015-11-25,2015-11-28,3,8
2015-12-02,2015-12-06,4,6


In [18]:
lateness['lateness'].mean()

3.25

In [19]:
def in_cycle_views(srs):
    """
    Given a Series in the `cycle_views` DataFrame, returns a dict of views for days in that story's news cycle.
    
    For example support a story was 3 days late and had only a 2 day news cycle, then the output should be:
    
    {'3': 333, '4': 444}
    """
    pub_date = srs.name[0]
    lag = lateness.ix[pub_date]['lateness']
    news_cycle_length = lateness.ix[pub_date]['news cycle length']
    return {day: srs[day] for day in range(lag, lag + news_cycle_length)}

# TODO: Augment the data table with information on views highly prior to publication and the highly posterior.
# This will standardize the "pre-cycle views" and "post-cycle views" metrics.

def pre_cycle_views(srs):
    """
    Given a Series in the `cycle_views` DataFrame, returns a dict of views for days prior to that story's news cycle.
    """
    return {day: srs[day] for day in range(0, min(in_cycle_views(srs).keys()))}

def post_cycle_views(srs):
    """
    Given a Series in the `cycle_views` DataFrame, returns a dict of views for days following the story's news cycle.
    """
    return {day: srs[day] for day in range(max(in_cycle_views(srs).keys()) + 1, 15)}

# cycle_frame['in-cycle total']
# cycle_frame.apply(axis=1, )

In [35]:
pre_cycle_views_list = []
in_cycle_views_list = []
post_cycle_views_list = []
# itterrows() zips the Series in a DataFrame into ((multiindex_one, multiindex_two), Series).
# Thus to retrieve the actual series we need series_repr[1].
# The operations defined above take over from there.
for series_repr in frame.iterrows():
    pre_cycle = pre_cycle_views(series_repr[1])
    pre_cycle_views_list.append(pre_cycle)
    in_cycle = in_cycle_views(series_repr[1])
    in_cycle_views_list.append(in_cycle)
    post_cycle = post_cycle_views(series_repr[1])
    post_cycle_views_list.append(post_cycle)
# At this point each of the lists has been populated with dictionaries that can be passed to a Series constructor.
# e.g. [{0: 16.0, 1: 3.0, 2: 2.0, 3: 8.0},
#       {0: 6.0, 1: 11.0, 2: 8.0, 3: 3.0},
#       ...]
# For cycle_view we will just compress this into a total.
pre_cycle_views_list = [sum(Series(dict_of_views)) for dict_of_views in pre_cycle_views_list]
in_cycle_views_list = [sum(Series(dict_of_views)) for dict_of_views in in_cycle_views_list]
post_cycle_views_list = [sum(Series(dict_of_views)) for dict_of_views in post_cycle_views_list]
out_of_cycle_views_list = [pre_cycle_views_list[i] + post_cycle_views_list[i] for i in range(0, len(in_cycle_views_list))]

In [42]:
cycle_frame = DataFrame(index=frame.index)
# cycle_frame['pre-cycle total'] = pre_cycle_views_list
cycle_frame['in-cycle total'] = in_cycle_views_list
# cycle_frame['post-cycle total'] = post_cycle_views_list
cycle_frame['out-of-cycle total'] = out_of_cycle_views_list
cycle_frame['overall total'] = frame['total']
cycle_frame['out-of-cycle percentage'] = cycle_frame.apply(lambda s: float('{0:.3f}'.format(s['out-of-cycle total'] / (s['out-of-cycle total'] + s['in-cycle total']))), axis=1)
# cycle_frame['pre-cycle percentage'] = cycle_frame.apply(lambda s: float('{0:.3f}'.format(s['pre-cycle total'] / (s['pre-cycle total'] + s['in-cycle total'] + s['post-cycle total']))), axis=1)
cycle_frame['in-cycle percentage'] = cycle_frame.apply(lambda s: float('{0:.3f}'.format(s['in-cycle total'] / (s['overall total']))), axis=1)
# cycle_frame['post-cycle percentage'] = cycle_frame.apply(lambda s: float('{0:.3f}'.format(s['post-cycle total'] / (s['pre-cycle total'] + s['in-cycle total'] + s['post-cycle total']))), axis=1)
cycle_frame = cycle_frame[['in-cycle total', 'in-cycle percentage', 'out-of-cycle total', 'out-of-cycle percentage', 'overall total']]

**How are *Signpost* views distributed with respect to publication status?**

The measure of the length of stories' news cycles that we created above gives us the tools we need in order to be able to answer this question, and after some munging (a lot of munging - it's difficult to account for publication lag periods) we are able to split up our totals into two parts:

* In-cycle views, which occur while the article is in the current news cycle (that is, the current issue). This is when we expect almost all views should occur, but we were surprised to learn that this is not always the case.
* Out-of-cycle views, which (mostly) occur when the article is no longer in the current news cycle (that is, it is now in an archived issue) or (sometimes) when it is still in the draft stage.

The chart below breaks *Signpost* article views up based on these two categories.

In [43]:
cycle_frame

in-cycle total  \
date       title                                                                                                                                                              
2015-10-07 Walled gardens of corruption                                                                                                                                2199   
           Tech news in brief                                                                                                                                           822   
           Reality is for losers                                                                                                                                       1085   
2015-10-14 Third Wikimedia Spain conference takes place in Madrid                                                                                                       797   
           Why the news media needs a Wikipedian in residence                                                                                                          1190   
           A fistful of dollars                                                                                                                                         911   
           2015–2016 Q1 fundraising update sparks mailing list debate                                                                                                  1190   
           WikiConference USA 2015: built on good faith                                                                                                                 968   
           Tech news in brief                                                                                                                                           745   
           Screens, sport, Reddit, and death                                                                                                                            924   
           US gathering sees speeches from Andrew Lih, AfroCrowd, and the Archivist of the United States                                                                996   
2015-10-21 Four ArbCom cases ongoing                                                                                                                                   1181   
           Women and Wikipedia: the world is watching                                                                                                                  3723   
           A more balanced week                                                                                                                                        1062   
           "Wikipedia's hostility to women"                                                                                                                            1669   
           Wikimedia lawsuit against NSA dismissed; Affiliates mailing list launched                                                                                   1336   
           Wikipedia is significantly amplifying the impact of open-access publications                                                                                1222   
           One year of GamerGate, or how I learned to stop worrying and love bare rule-level consensus                                                                 1522   
           Tech news in brief                                                                                                                                           951   
           Hiding under the covers of the Internet                                                                                                                     1245   
2015-10-28 A second attempt at Arbitration enforcement                                                                                                                  996   
           Five million articles                                                                                                    

**How many in-cycle views do *Signpost* stories generate, on average?**

Unsurprisingly, the majority of pagviews occur in-cycle.

In [44]:
print(int(cycle_frame['in-cycle total'].mean()), '{0:.3f}'.format(cycle_frame['in-cycle percentage'].mean()) + "%")

1237 0.791%


**How many views occur out-of-cycle?**

A not inconsiderable amount occur before or after, however (mostly after).

In [46]:
print(int(cycle_frame['out-of-cycle total'].mean()), '{0:.3f}'.format(cycle_frame['out-of-cycle percentage'].mean()) + "%")

312 0.209%
